In [1]:
# import the necessary packages

from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np

In [2]:
pip install split-folders tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
import splitfolders
import os

splitfolders.ratio('E:/Data Science and ML/Mask detector/dataset 2/Face-Mask-Detection-master/dataset/', output='hasil_split', seed=1337, ratio=(.6, .4))
hasil = 'hasil_split'
train_dir = os.path.join(hasil, 'train')
validation_dir = os.path.join(hasil, 'val')

train_withmask_dir = os.path.join(train_dir, 'with_mask')
train_withoutmask_dir = os.path.join(train_dir, 'without_mask')


val_withmask_dir = os.path.join(validation_dir, 'with_mask')
val_withoutmask_dir = os.path.join(validation_dir, 'without_mask')


Copying files: 3833 files [02:12, 28.85 files/s]


In [13]:
#TRAINING_DIR = "E:/Data Science/Mask detector/observations-master/experiements/dest_folder/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    color_mode="rgb",
                                                    batch_size=32, 
                                                    target_size=(150, 150))
#VALIDATION_DIR = "E:/Data Science/Mask detector/observations-master/experiements/dest_folder/test"
test_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

validation_generator = test_datagen.flow_from_directory(validation_dir, 
                                                        color_mode="rgb",
                                                         batch_size=32, 
                                                         target_size=(150, 150))

Found 2887 images belonging to 2 classes.
Found 1979 images belonging to 2 classes.


In [24]:
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])

opt = Adam(lr=1e-4, decay=1e-4 / 32)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['acc'])

In [25]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [26]:
history = model.fit_generator(train_generator,
                              epochs=10,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Epoch 1/10
91/91 [==============================] - ETA: 0s - loss: 0.4852 - acc: 0.7558

c:\users\asus\appdata\local\programs\python\python38\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model2-001.model\assets
91/91 [==============================] - 722s 8s/step - loss: 0.4852 - acc: 0.7558 - val_loss: 0.3393 - val_acc: 0.8757
Epoch 2/10
91/91 [==============================] - 2449s 27s/step - loss: 0.3133 - acc: 0.8791 - val_loss: 0.3208 - val_acc: 0.8772
Epoch 3/10
91/91 [==============================] - 669s 7s/step - loss: 0.3075 - acc: 0.8791 - val_loss: 0.2994 - val_acc: 0.8903
Epoch 4/10
91/91 [==============================] - 639s 7s/step - loss: 0.2956 - acc: 0.8826 - val_loss: 0.2757 - val_acc: 0.8959
Epoch 5/10
91/91 [==============================] - 655s 7s/step - loss: 0.2811 - acc: 0.8978 - val_loss: 0.2808 - val_acc: 0.8974
Epoch 6/10
91/91 [==============================] - 677s 7s/s

In [8]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
model=load_model("E:\Data Science and ML\Mask detector\model2-008.model")

results={0:'without mask',1:'use mask'}
GR_dict={0:(0,0,255),1:(0,255,0)}

rect_size = 4
cap = cv2.VideoCapture(0) 


haarcascade = cv2.CascadeClassifier('E:\Data Science and ML\Mask detector\opencv\opencv-master\data\haarcascades\haarcascade_frontalface_default.xml')

while True:
    (rval, im) = cap.read()
    im=cv2.flip(im,1,1) 

    
    rerect_size = cv2.resize(im, (im.shape[1] // rect_size, im.shape[0] // rect_size))
    faces = haarcascade.detectMultiScale(rerect_size)
    for f in faces:
        (x, y, w, h) = [v * rect_size for v in f] 
        
        face_img = im[y:y+h, x:x+w]
        rerect_sized=cv2.resize(face_img,(150,150))
        normalized=rerect_sized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)

        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),GR_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(im, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    
    if key == 27: 
        break

cap.release()

cv2.destroyAllWindows()